In [36]:
from google.colab import files
files.upload()

Saving After_Scrapping&Labelling.xlsx to After_Scrapping&Labelling.xlsx


{'After_Scrapping&Labelling.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee\x9dh^\x01\x00\x00\x90\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [ ]:
import pandas as pd
# !pip install english-words
from english_words import english_words_set

import numpy as nt
import os, csv, nltk
import pandas as pd
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import PunktSentenceTokenizer,RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# from gensim.models import ldamodel as lda

# !pip install lda
import os, csv, nltk, lda

In [38]:
def string_clean(s):
  s = s.replace("!@#$%^&*()[]{};:,./<>?\|`~-=_+", " ")
  list_of_words = s.split()
  
  list_return = []
  for x in list_of_words:
    if x.lower() in english_words_set and len(x)>2:
      list_return.append(x)
  
  return ' '.join(list_return)

In [39]:
df = pd.read_excel('After_Scrapping&Labelling.xlsx')
df['Caption_clean'] = df['Caption'].map(string_clean)

In [40]:
df.head()

,Unnamed: 0,Labels,URL,Caption,Likes,URL.1,source,Caption_clean
0,0,Smile Sleeve Font Happy Job Poster Wrinkle Eve...,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Every day a New Yorker dies waiting for a tran...,29,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,nycgov,Every day New for Help change that organ
1,1,Skyscraper Atmosphere Daytime Building Cloud W...,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Cover up New Yorkâ€™s beauty?\r\n\r\nFOG-getta...,432,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,nycgov,Cover New
2,2,Water Sky Building Cloud Font Tower block Real...,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,This might be the most perfect day of the year...,221,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,nycgov,This might the most perfect day the year ride ...
3,3,Cloud Sky Building Window Nature Plant Orange ...,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Sunset #intheBronx.,437,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,nycgov,Sunset
4,4,Water Water resources Plant World Asphalt Natu...,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Climate change is happening NOW. \r\n\r\nThe N...,278,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,nycgov,Climate change The New Extreme Weather New Cit...


In [37]:
df['Caption_clean'] = df['Caption'].map(string_clean)

## Topic Modelling Caption and Labels Combined

In [55]:
word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))

df['label_captions'] = df['Labels'] +' '+ df['Caption_clean'] 
print(df.loc[0]['label_captions'])

df.dropna(inplace=True)

def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

df['tokens'] = df['label_captions'].apply(tokenize_text)
df['tokens'] = df['tokens'].apply(lambda x: str(x)).apply(tokenize_text)

Smile Sleeve Font Happy Job Poster Wrinkle Event Brand Collar Every day New for Help change that organ


In [57]:
vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(df['label_captions'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
model = lda.LDA(n_topics=5, n_iter=500, random_state=1)
model.fit(total_features_words)

topic_word = model.topic_word_
doc_topic = model.doc_topic_
doc_topic = pd.DataFrame(doc_topic)
df_lda = df.join(doc_topic)

insta = pd.DataFrame()
insta = insta.reset_index()
topics = pd.DataFrame(topic_word)
topics.columns = vec_words.get_feature_names()
topics1 = topics.transpose()

In [60]:
topics1
topics1.to_csv('topics1.csv') 
files.download('topics1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Topic Modelling Captions Only

In [62]:
word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))

df['label_captions'] = df['Caption_clean'] 
print(df.loc[0]['label_captions'])

df.dropna(inplace=True)

def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

df['tokens'] = df['label_captions'].apply(tokenize_text)
df['tokens'] = df['tokens'].apply(lambda x: str(x)).apply(tokenize_text)

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(df['label_captions'])

model = lda.LDA(n_topics=3, n_iter=500, random_state=1)
model.fit(total_features_words)

topic_word = model.topic_word_
doc_topic = model.doc_topic_
doc_topic = pd.DataFrame(doc_topic)
df_lda = df.join(doc_topic)

insta = pd.DataFrame()
insta = insta.reset_index()
topics = pd.DataFrame(topic_word)
topics.columns = vec_words.get_feature_names()
topics1 = topics.transpose()

topics1
topics1.to_csv('topics1.csv') 
files.download('topics1.csv')

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
INFO:lda:n_documents: 1937
INFO:lda:vocab_size: 1598
INFO:lda:n_words: 9175
INFO:lda:n_topics: 3
INFO:lda:n_iter: 500


Every day New for Help change that organ


INFO:lda:<0> log likelihood: -81065
INFO:lda:<10> log likelihood: -67062
INFO:lda:<20> log likelihood: -66165
INFO:lda:<30> log likelihood: -65660
INFO:lda:<40> log likelihood: -65448
INFO:lda:<50> log likelihood: -65286
INFO:lda:<60> log likelihood: -65212
INFO:lda:<70> log likelihood: -65173
INFO:lda:<80> log likelihood: -65073
INFO:lda:<90> log likelihood: -64819
INFO:lda:<100> log likelihood: -64914
INFO:lda:<110> log likelihood: -64794
INFO:lda:<120> log likelihood: -64722
INFO:lda:<130> log likelihood: -64717
INFO:lda:<140> log likelihood: -64584
INFO:lda:<150> log likelihood: -64428
INFO:lda:<160> log likelihood: -64492
INFO:lda:<170> log likelihood: -64423
INFO:lda:<180> log likelihood: -64306
INFO:lda:<190> log likelihood: -64277
INFO:lda:<200> log likelihood: -64324
INFO:lda:<210> log likelihood: -64198
INFO:lda:<220> log likelihood: -64252
INFO:lda:<230> log likelihood: -64149
INFO:lda:<240> log likelihood: -64138
INFO:lda:<250> log likelihood: -64104
INFO:lda:<260> log like

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>